In [2]:
Sys.setlocale(category = "LC_ALL", locale = "en_US.UTF-8")

[1] "en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/C"

# Dependencies

In [1]:
pacman::p_load(ggplot2)

# Functions

In [3]:
source("/Users/anabbi/git/ped_CapTCRseq/R/ggplot2_theme.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/color_schemes.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/Misc_functions.R")

# Paths

In [4]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"
gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

# Main

In [5]:
metadata <- read.csv(
    file = paste0(datapath, "INT_metadata_flow.csv"),
    header = T, stringsAsFactors = F, colClasses = c("cycle" = "character"), check.names = F
)


In [6]:
head(metadata)

,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,CM,...,cancergroup,agegroup,foldgroup,sample_id,Remission,Relapse,Transplant,CART,vital_status,LOF
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,<int>,...,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
1,CHP_319,01,1.4,0,1,HR ALL,1.4,0,NA,NA,...,Leukemia,< 2 yrs,Low,CHP_319_01,0,1,0,1,1,0
2,CHP_319,02,7.9,0,1,HR ALL,7.9,0,NA,NA,...,Leukemia,< 2 yrs,High,CHP_319_02,0,1,0,1,1,0
3,CHP_319,03,1.8,0,1,HR ALL,1.8,0,NA,NA,...,Leukemia,< 2 yrs,Low,CHP_319_03,0,1,0,1,1,0
4,CHP_319,04,1.5,0,1,HR ALL,1.5,0,NA,NA,...,Leukemia,< 2 yrs,Low,CHP_319_04,0,1,0,1,1,0
5,CHP_319,05,1.2,0,1,HR ALL,1.2,0,NA,NA,...,Leukemia,< 2 yrs,Low,CHP_319_05,0,1,0,1,1,0
6,CHP_320,01,8.9,0,6,AML,8.9,0,NA,NA,...,Leukemia,5-11 yrs,High,CHP_320_01,0,1,0,0,1,0


In [7]:
colnames(metadata)

[1] "Patient"           "cycle"             "FOLD"             
 [4] "ATC"               "Age"               "Disease_type"     
 [7] "TE_fold"           "CD3"               "EM"               
[10] "CM"                "Naïve"             "SCM"              
[13] "TE"                "EM%"               "CM%"              
[16] "Naïve%"            "SCM%"              "TE%"              
[19] "Naïve/TE"          "N+CM+SCM/TE+EM"    "PD-1"             
[22] "PDL1"              "LAG3"              "TIM3"             
[25] "CTLA4"             "PD1%"              "PDL1%"            
[28] "LAG3%"             "TIM3%"             "CTLA4%"           
[31] "Any NCR"           "PD1 + LAG3 + TIM3" "cancergroup"      
[34] "agegroup"          "foldgroup"         "sample_id"        
[37] "Remission"         "Relapse"           "Transplant"       
[40] "CART"              "vital_status"      "LOF"

In [8]:
# confirmed with David, remove those samples with NCR% > 100
metadata$`TIM3%`[metadata$`TIM3%` > 100] <- NA
metadata$`LAG3%`[metadata$`LAG3%` > 100] <- NA
metadata$`PD1%`[metadata$`PD1%` > 100] <- NA
metadata$`PDL1%`[metadata$`PDL1%` > 100] <- NA

## TRB

In [9]:
divstats <- read.csv(paste0(datapath, "capTCRseq/divstats_TRBCHP_updated_corrected.csv"),
    header = T, stringsAsFactors = F, row.names = 1
)

In [10]:
head(divstats[ divstats$sample_id == "CHP_407_04",])

,TRB,Reads,CPKR,Average_reads,VMR,Max_reads,Singletons,Doubletons,qD,Sample_Coverage,...,SE_Richeness,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,sample_id,basename,file
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
CHP_407-04-PBMC-DNA,569,69502,8.186815,122.1476,0.7336426,659,9,4,574.9624,0.9999468,...,12.38727,440.9874,442.8405,1.043615,370.1329,372.1093,1.636065,CHP_407_04,CHP_407-04-PBMC-DNA,batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_407-04-PBMC-DNA.txt
CHP_407-04_cfDNA,1,58,17.241379,58.0000,NA,58,0,0,1.0000,1.0000000,...,0.00000,1.0000,1.0000,0.000000,1.0000,1.0000,0.000000,CHP_407_04,CHP_407-04_cfDNA,remaining_batch_3_cfDNA_15April2021/mixcr/clones/CLONES_TRBCHP_407-04_cfDNA.txt


In [11]:
divstats$sampletype <- NA
divstats$sampletype[grepl("PBMC", divstats$basename)] <- "PBMC"
divstats$sampletype[grepl("cfDNA", divstats$basename)] <- "cfDNA"
divstats$sampletype[grepl("FFPE", divstats$basename)] <- "FFPE"

In [12]:
table(divstats$sampletype, useNA = "always")


cfDNA  FFPE  PBMC  <NA> 
  287    18   288     0 

In [13]:
divstats[grepl("CHP_323", rownames(divstats)),]

,TRB,Reads,CPKR,Average_reads,VMR,Max_reads,Singletons,Doubletons,qD,Sample_Coverage,...,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,sample_id,basename,file,sampletype
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
CHP_323-03_cfDNA,103,81588,1.262441,792.11650,1.5296185,7047,8,0,109.95861,0.9999263,...,43.423179,43.456085,0.15187858,31.051999,31.063441,0.1585361,CHP_323_03,CHP_323-03_cfDNA,batch3a9_cfDNA_10Jul2020/mixcr/clones/CLONES_TRBCHP_323-03_cfDNA.txt,cfDNA
CHP_323-03-PBMC-DNA,45,3175,14.173228,70.55556,0.6027869,220,1,0,45.00000,1.0000000,...,37.721974,37.986092,0.38027285,33.203530,33.543868,0.6519925,CHP_323_03,CHP_323-03-PBMC-DNA,batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_323-03-PBMC-DNA.txt,PBMC
CHP_323-04-PBMC-DNA,23,671,34.277198,29.17391,1.2769705,167,1,1,23.43169,0.9997989,...,13.183675,13.413889,0.59376924,8.985232,9.093612,0.6029626,CHP_323_04,CHP_323-04-PBMC-DNA,batch7_25Oct2019/mixcr/clones/CLONES_TRBCHP_323-04-PBMC-DNA.txt,PBMC
CHP_323-05-PBMC-DNA,153,2020,75.742574,13.20261,1.9254929,290,10,8,157.98588,0.9990015,...,81.753356,85.173688,2.53829992,32.669335,33.189941,2.5682322,CHP_323_05,CHP_323-05-PBMC-DNA,batch7_25Oct2019/mixcr/clones/CLONES_TRBCHP_323-05-PBMC-DNA.txt,PBMC
CHP_323-05_cfDNA,15,4938,3.037667,329.20000,1.1360081,1391,3,0,16.89609,0.9997766,...,8.670523,8.686386,0.09652094,6.804325,6.812334,0.1201723,CHP_323_05,CHP_323-05_cfDNA,remaining_batch_1_cfDNA_8April2021/mixcr/clones/CLONES_TRBCHP_323-05_cfDNA.txt,cfDNA


In [14]:
lfs <- divstats$sample_id[grepl("LFS", divstats$sample_id)]
normals <- divstats$sample_id[grepl("YO", divstats$sample_id) | grepl("MO", divstats$sample_id)]

In [15]:
table(divstats$sampletype, useNA = "always")


cfDNA  FFPE  PBMC  <NA> 
  287    18   288     0 

In [16]:
#merge div file separately for each sample type, then merge, just to capture all data regardless of available datatype
meta_div_cf <- merge(metadata, divstats[divstats$sampletype == "cfDNA",], by = "sample_id", all = FALSE)
meta_div_pbmc <- merge(metadata, divstats[divstats$sampletype == "PBMC",], by = "sample_id", all = TRUE)

In [17]:
dim(metadata)
dim(meta_div_pbmc)
dim(divstats[divstats$sampletype == "PBMC",])

[1] 595  42

[1] 621  64

[1] 288  23

In [18]:
meta_div_cf[ meta_div_cf$sample_id == "CHP_407_01",]

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,SE_Richeness,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,basename,file,sampletype
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
249,CHP_407_01,CHP_407,01,1,216.81,12,DLBCL,1,2219,735,...,0,4.314781,4.316694,0.03156677,4.006012,4.008145,0.02787398,CHP_407-01_cfDNA,remaining_batch_3_cfDNA_15April2021/mixcr/clones/CLONES_TRBCHP_407-01_cfDNA.txt,cfDNA


In [19]:
div_ffpe <- divstats[divstats$sampletype == "FFPE",]

div_ffpe$Patient <- gsub("_A", "", div_ffpe$sample_id)
div_ffpe$Patient <- gsub("_B", "", div_ffpe$Patient)

In [20]:
meta_div_ffpe <- cbind(metadata[ match(div_ffpe$Patient, metadata$Patient),], div_ffpe)

In [21]:
which(colnames(meta_div_ffpe) == "sample_id")

[1] 36 62

In [22]:
meta_div_ffpe <- meta_div_ffpe[, c(36,1:35,38:ncol(meta_div_ffpe)-1)]

In [23]:
meta_div_ffpe$sample_id.1 <- NULL

In [24]:
head(meta_div_ffpe)

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,SE_Richeness,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,basename,file,sampletype
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
126,CHP_344_01,CHP_344,01,0.0,247.80,6,BL,0.0,0,NA,...,0.000000,4.807224,4.810892,0.05326430,4.009203,4.012886,0.068738694,CHP_344_A-FFPE-DNA,batchFFPE_3Apr2020/mixcr/clones/CLONES_TRBCHP_344_A-FFPE-DNA.txt,FFPE
211,CHP_361_01,CHP_361,01,2.4,516.04,13,HD,2.4,2317,94,...,3.810522,104.524707,104.825759,0.45502335,83.093330,83.359522,0.675356761,CHP_361-FFPE-DNA_S1,batchFFPE_3Apr2020/mixcr/clones/CLONES_TRBCHP_361-FFPE-DNA_S1.txt,FFPE
226,CHP_364_01,CHP_364,01,2.3,877.68,16,OS,2.3,2411,33,...,0.000000,7.123768,7.136111,0.10078646,5.433764,5.443025,0.144449256,CHP_364-FFPE-DNA_S1,batchFFPE_3Apr2020/mixcr/clones/CLONES_TRBCHP_364-FFPE-DNA_S1.txt,FFPE
236,CHP_366_01,CHP_366,01,8.9,1113.50,14,HD,8.9,NA,243,...,0.000000,4.720236,4.725638,0.08553979,3.424802,3.427516,0.079762310,CHP_366-FFPE-DNA_S1,batchFFPE_3Apr2020/mixcr/clones/CLONES_TRBCHP_366-FFPE-DNA_S1.txt,FFPE
256,CHP_370_01,CHP_370,01,4.2,0.00,17,ALCL,4.2,592,19,...,0.000000,2.555199,2.555493,0.01328165,2.267262,2.267426,0.009076097,CHP_370-FFPE-DNA_S5,batchFFPE_3Apr2020/mixcr/clones/CLONES_TRBCHP_370-FFPE-DNA_S5.txt,FFPE
266,CHP_372_01,CHP_372,01,1.0,0.00,21,BL,1.0,328,16,...,1.462165,2.406266,2.411727,0.04685703,1.739144,1.739628,0.032705491,CHP_372-FFPE-DNA_S1,batchFFPE_3Apr2020/mixcr/clones/CLONES_TRBCHP_372-FFPE-DNA_S1.txt,FFPE


In [25]:
meta_div <- rbind(meta_div_cf, meta_div_pbmc, meta_div_ffpe)

In [26]:
dim(meta_div)

[1] 925  64

In [27]:
#create a group for LFS and non-LFS
meta_div$Group <- NA
meta_div$Group[grepl("LFS", meta_div$sample_id)] <- "LFS"
meta_div$Group[grepl("CHP", meta_div$sample_id)] <- "CHP"

In [28]:
#create sample type
meta_div$sampletype <- NA
meta_div$sampletype[grepl("PBMC", meta_div$basename)] <- "PBMC"
meta_div$sampletype[grepl("cfDNA", meta_div$basename)] <- "cfDNA"
meta_div$sampletype[grepl("FFPE", meta_div$basename)] <- "Tumor"
#sampletyoe is PBMC if basename is NA, they are flow data
meta_div$sampletype[is.na(meta_div$basename)] <- "PBMC"

In [29]:
# add Patient id for normals, lfs
meta_div$Patient[meta_div$sample_id %in% c(lfs, normals)] <-
       substr(
              meta_div$sample_id[meta_div$sample_id %in% c(lfs, normals)], 1,
              nchar(meta_div$sample_id[meta_div$sample_id %in% c(lfs, normals)]) - 3
       )


In [30]:
# few samples we sequenced but was not in David's file
meta_div$Patient[is.na(meta_div$Patient) & meta_div$sampletype != "Tumor"] <-
       substr(
              meta_div$sample_id[is.na(meta_div$Patient) & meta_div$sampletype != "Tumor"], 1,
              nchar(meta_div$sample_id[is.na(meta_div$Patient) & meta_div$sampletype != "Tumor"]) - 3
       )


In [31]:
# Patient id same as sampel_id for tumor samples
meta_div$Patient[is.na(meta_div$Patient)] <- meta_div$sample_id[is.na(meta_div$Patient)]


In [32]:
# overwrite cycle for consistency

meta_div$cycle <- NA
meta_div$cycle <- substr(
       meta_div$sample_id,
       nchar(meta_div$sample_id) - 1, nchar(meta_div$sample_id)
)


In [33]:
#add X to prevent numeric
meta_div$cycle <- paste0("X", meta_div$cycle)

In [34]:
# cycle for tumors are 00
meta_div$cycle[ which(meta_div$sampletype == "Tumor")] <- "X00"

In [35]:
table(meta_div$cycle)


X00 X01 X02 X03 X04 X05 
 42 183 184 176 172 168 

In [36]:
#Disease_type for normals = Normal and lfs = LFS
meta_div$Disease_type[ meta_div$sample_id %in% normals] <- "Normal"

meta_div$Disease_type[ meta_div$sample_id %in% lfs] <- "LFS"

In [37]:
table(meta_div$Disease_type, useNA = "always")


  ALCL    ALL    AML   ARMS     BL    BLL    CML  DLBCL   ERMS    EWS     HB 
    15    180     82      9     37     17     21     22     42     48     27 
    HD HR ALL    LFS  MS NB     NB Normal     OS  PMBCL    RMS SR ALL  T-ALL 
    67    105     10      6     29     14     93     16     15     18     31 
 WILMS   <NA> 
    19      2 

In [38]:
#minor fix
meta_div$Patient[ meta_div$Patient == "CHP_404_A"] <- "CHP_404"
meta_div$Patient[ meta_div$Patient == "CHP_404_B"] <- "CHP_404"
meta_div$Patient[ meta_div$Patient == "CHP_344_A"] <- "CHP_344"

In [39]:
unique(meta_div$Patient)

[1] "CHP_323"      "CHP_324"      "CHP_326"      "CHP_328"      "CHP_329"     
  [6] "CHP_330"      "CHP_331"      "CHP_333"      "CHP_334"      "CHP_335"     
 [11] "CHP_336"      "CHP_337"      "CHP_338"      "CHP_339"      "CHP_340"     
 [16] "CHP_341"      "CHP_342"      "CHP_343"      "CHP_344"      "CHP_345"     
 [21] "CHP_347"      "CHP_348"      "CHP_349"      "CHP_350"      "CHP_351"     
 [26] "CHP_353"      "CHP_355"      "CHP_356"      "CHP_357"      "CHP_358"     
 [31] "CHP_359"      "CHP_360"      "CHP_361"      "CHP_362"      "CHP_363"     
 [36] "CHP_364"      "CHP_365"      "CHP_366"      "CHP_367"      "CHP_368"     
 [41] "CHP_369"      "CHP_370"      "CHP_371"      "CHP_372"      "CHP_373"     
 [46] "CHP_374"      "CHP_375"      "CHP_376"      "CHP_377"      "CHP_378"     
 [51] "CHP_379"      "CHP_380"      "CHP_381"      "CHP_382"      "CHP_383"     
 [56] "CHP_384"      "CHP_385"      "CHP_386"      "CHP_387"      "CHP_388"     
 [61] "CHP_389"      "CHP_390"      "CHP_391"      "CHP_392"      "CHP_393"     
 [66] "CHP_394"      "CHP_395"      "CHP_396"      "CHP_397"      "CHP_398"     
 [71] "CHP_399"      "CHP_400"      "CHP_401"      "CHP_402"      "CHP_403"     
 [76] "CHP_404"      "CHP_405"      "CHP_407"      "CHP_408"      "CHP_409"     
 [81] "CHP_410"      "CHP_411"      "CHP_412"      "CHP_413"      "CHP_414"     
 [86] "CHP_415"      "CHP_416"      "CHP_417"      "CHP_418"      "CHP_419"     
 [91] "CHP_420"      "CHP_421"      "CHP_422"      "CHP_423"      "CHP_10YO_M"  
 [96] "CHP_11YO_M"   "CHP_15YO_F_A" "CHP_15YO_F_B" "CHP_15YO_M"   "CHP_16YO_M"  
[101] "CHP_1YO_M_A"  "CHP_1YO_M_B"  "CHP_2YO_M"    "CHP_303"      "CHP_315"     
[106] "CHP_319"      "CHP_320"      "CHP_321"      "CHP_322"      "CHP_325"     
[111] "CHP_327"      "CHP_332"      "CHP_346"      "CHP_352"      "CHP_354"     
[116] "CHP_3YO_F"    "CHP_3YO_M_A"  "CHP_3YO_M_B"  "CHP_406"      "CHP_424"     
[121] "CHP_425"      "CHP_426"      "CHP_427"      "CHP_428"      "CHP_429"     
[126] "CHP_430"      "CHP_431"      "CHP_432"      "CHP_433"      "CHP_434"     
[131] "CHP_435"      "CHP_436"      "CHP_437"      "CHP_5YO_M"    "CHP_8MO_F"   
[136] "LFS_TCR_4535" "LFS_TCR_4570" "LFS_TCR_4602" "LFS_TCR_4758" "LFS_TCR_4759"
[141] "LFS_TCR_4783" "LFS_TCR_4837" "LFS_TCR_4838" "LFS_TCR_5014" "LFS_TCR_5067"

In [40]:
# add dx for tumors
for (i in unique(meta_div$Patient[which(meta_div$sampletype == "Tumor")])) {
   # print(i)
   tmp <- meta_div[meta_div$Patient == i, ]
   cancer <- names(table(tmp$Disease_type)[1])
   meta_div$Disease_type[meta_div$Patient == i &
      meta_div$sampletype %in% "Tumor"] <- cancer
}


In [41]:
meta_div[ is.na(meta_div$Disease_type),]

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,basename,file,sampletype,Group
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
296,CHP_303_05,CHP_303,X05,NA,NA,NA,NA,NA,NA,NA,...,191.49643,194.71378,1.3610489,158.597414,161.725093,1.9785872,CHP_303-05-PBMC-DNA,batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_303-05-PBMC-DNA.txt,PBMC,CHP
297,CHP_315_05,CHP_315,X05,NA,NA,NA,NA,NA,NA,NA,...,15.46954,15.79328,0.6325886,9.026271,9.137733,0.6765911,CHP_315-05-PBMC-DNA,batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,PBMC,CHP


In [42]:
# old samples. Remove them!
meta_div <- meta_div[ !is.na(meta_div$Disease_type),]

In [43]:
table(meta_div$cancergroup, useNA = "always")


Leukemia Lymphoma    Solid     <NA> 
     437      174      288       24 

In [44]:
table(meta_div$Disease_type, useNA = "always")


  ALCL    ALL    AML   ARMS     BL    BLL    CML  DLBCL   ERMS    EWS     HB 
    15    180     82      9     37     17     21     22     42     48     27 
    HD HR ALL    LFS  MS NB     NB Normal     OS  PMBCL    RMS SR ALL  T-ALL 
    67    105     10      6     29     14     93     16     15     18     31 
 WILMS   <NA> 
    19      0 

In [45]:
meta_div$cancergroup[ meta_div$sampletype == "Tumor"] <- "Solid"
meta_div$cancergroup[ meta_div$Disease_type == "Normal"] <- "Normal"
meta_div$cancergroup[ meta_div$Disease_type == "LFS"] <- "LFS"

In [46]:
table(meta_div$cancergroup, useNA = "always")


Leukemia      LFS Lymphoma   Normal    Solid     <NA> 
     437       10      164       14      298        0 

In [47]:
table(meta_div$sampletype, useNA = "always")


cfDNA  PBMC Tumor  <NA> 
  286   619    18     0 

In [48]:
head(meta_div[meta_div$sample_id == "CHP_407_01",])

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,basename,file,sampletype,Group
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
249,CHP_407_01,CHP_407,X01,1,216.81,12,DLBCL,1,2219,735,...,4.314781,4.316694,0.03156677,4.006012,4.008145,0.02787398,CHP_407-01_cfDNA,remaining_batch_3_cfDNA_15April2021/mixcr/clones/CLONES_TRBCHP_407-01_cfDNA.txt,cfDNA,CHP
741,CHP_407_01,CHP_407,X01,1,216.81,12,DLBCL,1,2219,735,...,296.625795,328.340127,4.82121497,247.015293,280.442163,6.77665004,CHP_407-01_PBMC_DNA,batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_407-01_PBMC_DNA.txt,PBMC,CHP
4411,CHP_407_01,CHP_407,X00,1,216.81,12,DLBCL,1,2219,735,...,3.243773,3.244667,0.02276269,2.904515,2.905275,0.02163161,CHP_407-FFPE-DNA_S7,batchFFPE_3Apr2020/mixcr/clones/CLONES_TRBCHP_407-FFPE-DNA_S7.txt,Tumor,CHP


In [49]:
length(unique(meta_div$Patient))

[1] 143

In [50]:
meta_div[ meta_div$Patient == "CHP_407",]

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,observed_Shannon,estimated_Shannon,SE_Shannon,observed_Simpson,estimated_Simpson,SE_Simpson,basename,file,sampletype,Group
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
249,CHP_407_01,CHP_407,X01,1.0,216.81,12,DLBCL,1.0,2219,735,...,4.314781,4.316694,0.03156677,4.006012,4.008145,0.02787398,CHP_407-01_cfDNA,remaining_batch_3_cfDNA_15April2021/mixcr/clones/CLONES_TRBCHP_407-01_cfDNA.txt,cfDNA,CHP
250,CHP_407_02,CHP_407,X02,18.1,570.15,12,DLBCL,18.1,2373,839,...,6.629124,6.638905,0.09053794,4.708198,4.711226,0.08111305,CHP_407-02_cfDNA,priority_pool3/mixcr/clones/CLONES_TRBCHP_407-02_cfDNA.txt,cfDNA,CHP
251,CHP_407_03,CHP_407,X03,3.7,86.24,12,DLBCL,3.7,1185,72,...,1.481101,1.487815,0.05912977,1.230549,1.231358,0.03432500,CHP_407-03_cfDNA,batch11_cfDNA_11Sep2020/mixcr/clones/CLONES_TRBCHP_407-03_cfDNA.txt,cfDNA,CHP
252,CHP_407_04,CHP_407,X04,0.0,0.00,12,DLBCL,0.0,0,NA,...,1.000000,1.000000,0.00000000,1.000000,1.000000,0.00000000,CHP_407-04_cfDNA,remaining_batch_3_cfDNA_15April2021/mixcr/clones/CLONES_TRBCHP_407-04_cfDNA.txt,cfDNA,CHP
741,CHP_407_01,CHP_407,X01,1.0,216.81,12,DLBCL,1.0,2219,735,...,296.625795,328.340127,4.82121497,247.015293,280.442163,6.77665004,CHP_407-01_PBMC_DNA,batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_407-01_PBMC_DNA.txt,PBMC,CHP
742,CHP_407_02,CHP_407,X02,18.1,570.15,12,DLBCL,18.1,2373,839,...,40.144620,41.509305,0.91390958,32.776496,33.849983,1.02611973,CHP_407-02_PBMC_DNA,batch11_23Apr2020/mixcr/clones/CLONES_TRBCHP_407-02_PBMC_DNA.txt,PBMC,CHP
743,CHP_407_03,CHP_407,X03,3.7,86.24,12,DLBCL,3.7,1185,72,...,23.868713,24.224584,0.59816864,17.986145,18.256270,0.83346694,CHP_407-03__PBMC-DN,batch12_30Apr2021/mixcr/clones/CLONES_TRBCHP_407-03__PBMC-DN.txt,PBMC,CHP
744,CHP_407_04,CHP_407,X04,0.0,0.00,12,DLBCL,0.0,0,NA,...,440.987356,442.840531,1.04361494,370.132939,372.109282,1.63606502,CHP_407-04-PBMC-DNA,batch13_26Aug2020/mixcr/clones/CLONES_TRBCHP_407-04-PBMC-DNA.txt,PBMC,CHP
745,CHP_407_05,CHP_407,X05,0.0,0.00,12,DLBCL,0.0,0,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,PBMC,CHP


In [51]:
save(meta_div, file = paste0(datapath,"capTCRseq/meta_div_TRB.RData"))